In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from datetime import datetime as dt
import time
import math

In [2]:
#데이터 불러오기
user = pd.read_csv('data/user_spec.csv')
loan = pd.read_csv('data/loan_result.csv')

In [3]:
#데이터셋의 결측치 확인
print(user.isnull().sum())
print('------------------')
print(loan.isnull().sum())

application_id                               0
user_id                                      0
birth_year                               12961
gender                                   12961
insert_time                                  0
credit_score                            105115
yearly_income                               90
income_type                                 85
company_enter_month                     171760
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn              587461
personal_rehabilitation_complete_yn    1203354
existing_loan_cnt                       198556
existing_loan_amt                       313774
dtype: int64
------------------
application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                  7495
loan

In [4]:
#입사일을 계산하기 위하여 형태 변환 후 월까지의 데아터만 잘라옴
user['company_enter_month'] = user['company_enter_month'].astype(str)
user['enter_month']=user['company_enter_month'].str[:6]

In [5]:
#입사년도를 계산하기 위하여 칼럼추가
user['year']=202206
user.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0,201511,202206
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,200702,202206
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0,202109,202206
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0,201701,202206
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0,202109,202206


In [6]:
#입사년도의 데이터를 평균값으로 나타낼시, 오류가 발생함으로 이 데이터의 결측치는 드롭해줌
user['enter_month'] = user['enter_month'].astype(float)
user=user.dropna(subset=['enter_month'])
user.head(5)

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0,201511.0,202206
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,200702.0,202206
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0,202109.0,202206
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0,201701.0,202206
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0,202109.0,202206


In [7]:
#나이를 계산하기 위하여 칼럼추가
user['age_year']=2022

In [8]:
user['enter_month'] = user['enter_month'].astype(int)
user.head(5)

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year,age_year
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0,201511,202206,2022
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,200702,202206,2022
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0,202109,202206,2022
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0,201701,202206,2022
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0,202109,202206,2022


In [9]:
#입사일을 계산하기 위해서 형식 변환
user['year'] = pd.to_datetime(user['year'],format="%Y%m")
user['enter_month'] = pd.to_datetime(user['enter_month'],format="%Y%m")

In [10]:
user['work_day'] =user['year'] -  user['enter_month']
user.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year,age_year,work_day
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,...,1000000.0,기타,0.0,NaN,4.0,162000000.0,2015-11-01,2022-06-01,2022,2404 days
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,2007-02-01,2022-06-01,2022,5599 days
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,...,10000000.0,생활비,0.0,NaN,5.0,15000000.0,2021-09-01,2022-06-01,2022,273 days
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,...,2000000.0,생활비,0.0,NaN,7.0,344000000.0,2017-01-01,2022-06-01,2022,1977 days
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,...,5000000.0,생활비,0.0,0.0,1.0,16000000.0,2021-09-01,2022-06-01,2022,273 days


In [11]:
user.corr()

,application_id,user_id,birth_year,gender,credit_score,yearly_income,desired_amount,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,age_year
application_id,1.000000,-0.001075,-0.000184,0.000147,-0.000304,0.000910,0.000176,0.000356,-0.002196,-0.000495,0.001346,NaN
user_id,-0.001075,1.000000,0.003234,0.005476,0.004832,0.007938,0.001949,0.001265,0.004728,-0.002576,0.002010,NaN
birth_year,-0.000184,0.003234,1.000000,0.045157,-0.038677,-0.067752,-0.021078,-0.021316,-0.068572,-0.090865,-0.164065,NaN
gender,0.000147,0.005476,0.045157,1.000000,0.019778,0.033287,0.013625,-0.003011,0.007750,0.005696,0.097289,NaN
credit_score,-0.000304,0.004832,-0.038677,0.019778,1.000000,0.041116,0.084190,-0.191753,-0.083116,-0.130988,0.164336,NaN
yearly_income,0.000910,0.007938,-0.067752,0.033287,0.041116,1.000000,0.125402,-0.004319,0.002967,0.009402,0.115568,NaN
desired_amount,0.000176,0.001949,-0.021078,0.013625,0.084190,0.125402,1.000000,-0.000560,0.007228,-0.026095,0.048935,NaN
personal_rehabilitation_yn,0.000356,0.001265,-0.021316,-0.003011,-0.191753,-0.004319,-0.000560,1.000000,0.314795,-0.053946,-0.046056,NaN
personal_rehabilitation_complete_yn,-0.002196,0.004728,-0.068572,0.007750,-0.083116,0.002967,0.007228,0.314795,1.000000,-0.039461,-0.022540,NaN
existing_loan_cnt,-0.000495,-0.002576,-0.090865,0.005696,-0.130988,0.009402,-0.026095,-0.053946,-0.039461,1.000000,0.270385,NaN


In [12]:
user['birth_year'].fillna(user['birth_year'].mean(), inplace = True)
user['credit_score'].fillna(user['credit_score'].mean(), inplace = True)
user['yearly_income'].fillna(user['yearly_income'].mean(), inplace = True)

In [13]:
user["income_type"].fillna(user["income_type"].mode(),inplace=True)
user["employment_type"].fillna(user["employment_type"].mode(),inplace=True)
user["houseown_type"].fillna(user["houseown_type"].mode(),inplace=True)
user["purpose"].fillna(user["purpose"].mode(),inplace=True)

In [14]:
#성별 
user.gender.fillna(user.gender.median(),inplace=True)

In [15]:
#나이계산
user['Age']=user['age_year']-user['birth_year']
user.head(10)

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year,age_year,work_day,Age
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,...,기타,0.0,NaN,4.0,162000000.0,2015-11-01,2022-06-01,2022,2404 days,37.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,대환대출,0.0,NaN,1.0,27000000.0,2007-02-01,2022-06-01,2022,5599 days,54.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,...,생활비,0.0,NaN,5.0,15000000.0,2021-09-01,2022-06-01,2022,273 days,25.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,...,생활비,0.0,NaN,7.0,344000000.0,2017-01-01,2022-06-01,2022,1977 days,33.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,...,생활비,0.0,0.0,1.0,16000000.0,2021-09-01,2022-06-01,2022,273 days,22.0
5,1559350,746993,1994.0,1.0,2022-06-07 09:55:03,600.0,35000000.0,FREELANCER,20160401.0,기타,...,생활비,0.0,NaN,1.0,NaN,2016-04-01,2022-06-01,2022,2252 days,28.0
7,1146166,588743,1992.0,0.0,2022-06-07 17:19:33,750.0,46000000.0,EARNEDINCOME,20151001.0,정규직,...,생활비,0.0,0.0,NaN,NaN,2015-10-01,2022-06-01,2022,2435 days,30.0
8,1153613,578866,1991.0,1.0,2022-06-07 14:55:49,560.0,38000000.0,EARNEDINCOME,20181101.0,정규직,...,대환대출,0.0,NaN,5.0,38000000.0,2018-11-01,2022-06-01,2022,1308 days,31.0
9,311143,646289,2000.0,1.0,2022-06-07 10:17:19,530.0,28000000.0,EARNEDINCOME,20220601.0,계약직,...,생활비,0.0,0.0,5.0,19000000.0,2022-06-01,2022-06-01,2022,0 days,22.0
10,1607787,555995,1980.0,1.0,2022-06-07 09:47:01,640.0,57000000.0,EARNEDINCOME,20220501.0,계약직,...,대환대출,0.0,0.0,4.0,62000000.0,2022-05-01,2022-06-01,2022,31 days,42.0


In [16]:
# 자기회생여부 전처리
# 1. personal_rehabilitation_yn == 0 -> personal_rehabilitation_complete_yn = 0
# 2. personal_rehabilitation_complete_yn == 1 ->  personal_rehabilitation_yn == 1
# 3. personal_rehabilitation_yn == 0 인데 personal_rehabilitation_complete_yn == 1이면 제거 => 1번 전처리로 처리됨.

# 1
indexs = user[(user['personal_rehabilitation_yn'] == 0 ) & (user['personal_rehabilitation_complete_yn'] != 0)].index
user.loc[indexs, ['personal_rehabilitation_complete_yn']] = 0

# 2
indexs_2 = user[(user['personal_rehabilitation_complete_yn'] == 1 ) & (user['personal_rehabilitation_yn'] != 1)].index
indexs_2
user.loc[indexs_2, ['personal_rehabilitation_yn']] = 1

user[user['personal_rehabilitation_yn'].isnull()]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year,age_year,work_day,Age
47,1349293,521056,1995.0,0.0,2022-06-07 16:57:35,560.00000,30000000.0,PRIVATEBUSINESS,20200701.0,정규직,...,대환대출,NaN,NaN,4.0,28000000.0,2020-07-01,2022-06-01,2022,700 days,27.0
102,865393,302309,1991.0,0.0,2022-06-07 07:44:15,620.00000,36000000.0,EARNEDINCOME,20210801.0,정규직,...,대환대출,NaN,NaN,NaN,NaN,2021-08-01,2022-06-01,2022,304 days,31.0
148,591548,147098,1985.0,0.0,2022-06-07 13:21:16,630.00000,54000000.0,EARNEDINCOME,20210601.0,정규직,...,생활비,NaN,NaN,NaN,NaN,2021-06-01,2022-06-01,2022,365 days,37.0
155,1617530,379488,1976.0,1.0,2022-04-20 10:08:40,600.00000,100000000.0,PRIVATEBUSINESS,201210.0,정규직,...,생활비,NaN,NaN,1.0,14000000.0,2012-10-01,2022-06-01,2022,3530 days,46.0
159,1232849,575347,1982.0,1.0,2022-04-20 11:49:52,620.00000,48000000.0,EARNEDINCOME,202202.0,계약직,...,생활비,NaN,NaN,1.0,NaN,2022-02-01,2022-06-01,2022,120 days,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394210,316356,53701,1996.0,1.0,2022-03-22 10:01:38,730.00000,60000000.0,PRIVATEBUSINESS,202105.0,정규직,...,사업자금,NaN,NaN,1.0,NaN,2021-05-01,2022-06-01,2022,396 days,26.0
1394211,1864587,489900,2000.0,1.0,2022-03-22 14:55:32,590.00000,25000000.0,FREELANCER,202106.0,기타,...,사업자금,NaN,NaN,NaN,NaN,2021-06-01,2022-06-01,2022,365 days,22.0
1394213,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.00000,75000000.0,EARNEDINCOME,200908.0,정규직,...,대환대출,NaN,NaN,8.0,200000000.0,2009-08-01,2022-06-01,2022,4687 days,39.0
1394214,1482466,766546,1975.0,1.0,2022-03-22 22:12:35,640.00000,50000000.0,EARNEDINCOME,201705.0,정규직,...,대환대출,NaN,NaN,10.0,117000000.0,2017-05-01,2022-06-01,2022,1857 days,47.0


In [17]:
# 위에 텍스트 읽고 실행
indexs_3 = user[(user['personal_rehabilitation_yn'].isnull()) & (user['personal_rehabilitation_complete_yn'].isnull()) ].index
user.loc[indexs_3, ['personal_rehabilitation_yn']] = 0
user.loc[indexs_3, ['personal_rehabilitation_complete_yn']] = 0

user.isnull().sum()

application_id                              0
user_id                                     0
birth_year                                  0
gender                                      0
insert_time                                 0
credit_score                                0
yearly_income                               0
income_type                                 0
company_enter_month                         0
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn                  0
personal_rehabilitation_complete_yn         0
existing_loan_cnt                      168327
existing_loan_amt                      266036
enter_month                                 0
year                                        0
age_year                                    0
work_day                                    0
Age                               

In [18]:
indexs_1 = user[(user['existing_loan_cnt'].isnull()) & (user['existing_loan_amt'].isnull()) ].index
user.loc[indexs_1, ['existing_loan_cnt']] = 0
user.loc[indexs_1, ['existing_loan_amt']] = 0

In [19]:
df1 = user.loc[user['existing_loan_cnt'] == 1 & user['existing_loan_amt'].notnull()]
mean = user['existing_loan_amt'].describe()['mean']

user['existing_loan_amt'].fillna(math.floor(mean), inplace = True)

In [20]:
df1.isnull().sum()

application_id                         0
user_id                                0
birth_year                             0
gender                                 0
insert_time                            0
credit_score                           0
yearly_income                          0
income_type                            0
company_enter_month                    0
employment_type                        0
houseown_type                          0
desired_amount                         0
purpose                                0
personal_rehabilitation_yn             0
personal_rehabilitation_complete_yn    0
existing_loan_cnt                      0
existing_loan_amt                      0
enter_month                            0
year                                   0
age_year                               0
work_day                               0
Age                                    0
dtype: int64

In [21]:
purpose = {'purpose' : {'ETC':'기타', 'INVEST':'투자','LIVING':'생활비','BUSINESS':'사업자금','BUYCAR':'자동차구입','BUYHOUSE':'주택구입','SWITCHLOAN':'대환대출','HOUSEDEPOSIT':'전월세보증금'}}
df1 = df1.replace(purpose)
df1['purpose'].unique()

array(['대환대출', '생활비', '주택구입', '기타', '사업자금', '전월세보증금', '자동차구입', '투자'],
      dtype=object)

In [22]:
##데이터 합쳐주기


In [23]:
loan = pd.read_csv('loan_result_clean.csv')
user = pd.read_csv('user_spec_clean.csv')

In [24]:
user = user.drop('Unnamed: 0', axis = 1)
user.head(5)

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_month,year,age_year,work_day,Age
0,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,대환대출,0.0,0.0,1.0,27000000.0,2007-02-01,2022-06-01,2022,5599 days,54.0
1,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,...,생활비,0.0,0.0,1.0,16000000.0,2021-09-01,2022-06-01,2022,273 days,22.0
2,2003486,441096,1987.0,0.0,2022-06-07 15:34:07,600.0,75000000.0,FREELANCER,20210301.0,기타,...,생활비,0.0,0.0,1.0,7000000.0,2021-03-01,2022-06-01,2022,457 days,35.0
3,1018194,50628,1985.0,1.0,2022-06-07 11:25:04,640.0,43000000.0,EARNEDINCOME,20180301.0,정규직,...,생활비,0.0,0.0,1.0,8000000.0,2018-03-01,2022-06-01,2022,1553 days,37.0
4,1097135,57840,1968.0,0.0,2022-06-07 23:13:06,560.0,156000000.0,EARNEDINCOME,20150101.0,정규직,...,생활비,0.0,0.0,1.0,5000000.0,2015-01-01,2022-06-01,2022,2708 days,54.0


In [25]:
loan = loan.drop('Unnamed: 0', axis = 1)
loan.head(5)

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
